In [2]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from model import LSTMAutoencoder
from preprocessing import load_data, normalize_data, segment_windows

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def main():
    # Load and preprocess data
    df = load_data()
    # If your data has a label column, drop it for unsupervised training
    if 'label' in df.columns:
        df = df[df['label'] == 0]  # Train only on normal data
        df = df.drop(columns=['label'])
    data_norm = normalize_data(df)
    windows = segment_windows(data_norm, window_size=50, step_size=10)

    # Convert to torch tensors
    X = torch.tensor(windows, dtype=torch.float32)

    # Model parameters
    seq_len = X.shape[1]
    n_features = X.shape[2]
    embedding_dim = 64

    model = LSTMAutoencoder(seq_len, n_features, embedding_dim).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.MSELoss()

    # Training
    n_epochs = 30
    batch_size = 32
    train_loader = DataLoader(TensorDataset(X), batch_size=batch_size, shuffle=True)

    model.train()
    for epoch in range(n_epochs):
        epoch_loss = 0
        for batch in train_loader:
            batch_x = batch[0].to(DEVICE)
            output = model(batch_x)
            loss = criterion(output, batch_x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")

    # Save model
    torch.save(model.state_dict(), "../data/lstm_autoencoder.pth")
    print("Model saved to ../data/lstm_autoencoder.pth")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'model'